### RSI _Opposite_Order ( RSI_O2 ) Strategy

In [1]:
import time
from datetime import datetime

import pandas as pd
import pandas_ta as ta
import numpy as np

### import csv

In [2]:
data = pd.read_csv('AUDUSD_15m_2022_01_03.csv')
data.tail()

,timestamp,open,high,low,close,adj close,volumn
3895,2021-12-31 21:15:00+00:00,0.727151,0.727431,0.727104,0.727310,0.727310,0
3896,2021-12-31 21:30:00+00:00,0.727299,0.727431,0.726903,0.727061,0.727061,0
3897,2021-12-31 21:45:00+00:00,0.726966,0.727590,0.726860,0.726908,0.726908,0
3898,2021-12-31 22:00:00+00:00,0.726216,0.726375,0.726216,0.726375,0.726375,0
3899,2021-12-31 22:15:00+00:00,0.726375,0.726375,0.726375,0.726375,0.726375,0


### create strategy

In [3]:
rsi_o2 = ta.Strategy(
    name = 'RSI 14 Days for opposite trade',
    description = 'RSI for alert',
    ta = [
        {'kind': 'ema', 'length': 200},
        {'kind': 'rsi', 'length': 24},
        {'kind': 'atr', 'length': 14}
    ]
)

In [4]:
df = data.copy()
df.ta.strategy(rsi_o2)
df.tail()

,timestamp,open,high,low,close,adj close,volumn,EMA_200,RSI_24,ATRr_14
3895,2021-12-31 21:15:00+00:00,0.727151,0.727431,0.727104,0.727310,0.727310,0,0.725800,54.033118,0.000617
3896,2021-12-31 21:30:00+00:00,0.727299,0.727431,0.726903,0.727061,0.727061,0,0.725812,52.158400,0.000610
3897,2021-12-31 21:45:00+00:00,0.726966,0.727590,0.726860,0.726908,0.726908,0,0.725823,51.018862,0.000619
3898,2021-12-31 22:00:00+00:00,0.726216,0.726375,0.726216,0.726375,0.726375,0,0.725829,47.269766,0.000624
3899,2021-12-31 22:15:00+00:00,0.726375,0.726375,0.726375,0.726375,0.726375,0,0.725834,47.269766,0.000579


### create signal

In [5]:
def backtest_rsi_o2(df, ema_len=200, rsi_len=14, atr_len=14, overbought=70, oversold=30, rr=1.5, pip_value=0.0001):
    
    rsi_o2 = ta.Strategy(
        name = 'RSI 14 Days for opposite trade',
        description = 'RSI for alert',
        ta = [
            {'kind': 'ema', 'length': ema_len},
            {'kind': 'rsi', 'length': rsi_len},
            {'kind': 'atr', 'length': atr_len}
        ]
    )
    
    df.ta.strategy(rsi_o2)
    df['action'] = np.nan

    orders = pd.DataFrame(columns = ['action', 'open time', 'open', 'close time', 'close', 'T/P', 'S/L','result'])
    current_action = 'close'
    order = {}
    unknow_result = []

    for index, row in df.iterrows():
        if current_action == 'close':
            
            if row['EMA_'+str(ema_len)] < row['low'] and row['RSI_'+str(rsi_len)] > overbought:
                row['action'] = 'buy'
                order['action'] = 'buy'
                order['open time'] = row['timestamp']
                order['open'] = row['open']
                order['S/L'] = row['low'] - row['ATRr_'+str(atr_len)]
                order['T/P'] = row['open'] + (rr * abs(row['open'] - order['S/L']))
                current_action = 'buy'
            
            elif row['EMA_'+str(ema_len)] > row['high'] and row['RSI_'+str(rsi_len)] < oversold:   
                row['action'] = 'sell'
                order['action'] = 'sell'
                order['open time'] = row['timestamp']
                order['open'] = row['open']
                order['S/L'] = row['high'] + row['ATRr_'+str(atr_len)]
                order['T/P'] = row['open'] - (rr * abs(row['open'] - order['S/L']))
                current_action = 'sell'
 
        if current_action == 'buy':
            if row['low'] <= order['S/L'] and row['high'] >= order['T/P']:
                unknow_result.append(row)
            if row['low'] <= order['S/L']:
                order['result'] = 'S/L'
                order['close time'] = row['timestamp']
                order['close'] = order['S/L']
                current_action = 'close'
                orders = orders.append(order, ignore_index = True)
                order = {}
            elif row['high'] >= order['T/P']:
                order['result'] = 'T/P'
                order['close time'] = row['timestamp']
                order['close'] = order['T/P']
                current_action = 'close'
                orders = orders.append(order, ignore_index = True)
                order = {}
                
        elif current_action == 'sell':
            if row['high'] >= order['S/L'] and row['low'] <= order['T/P']:
                unknow_result.append(row)
            if row['high'] >= order['S/L']:
                order['result'] = 'S/L'
                order['close time'] = row['timestamp']
                order['close'] = order['S/L']
                current_action = 'close'
                orders = orders.append(order, ignore_index = True)
                order = {}
            elif row['low'] <= order['T/P']:
                order['result'] = 'T/P'
                order['close time'] = row['timestamp']
                order['close'] = order['T/P']
                current_action = 'close'
                orders = orders.append(order, ignore_index = True)
                order = {}

    try:
        win_rate = round(len(orders[orders['result'] == 'T/P']) * 100 / len(orders), 2)
        loss_rate = round(len(orders[orders['result'] == 'S/L']) * 100 / len(orders), 2)

        win_orders = int((win_rate * len(orders)) / 100 )
        loss_orders = len(orders) - win_orders
        gain = (win_orders * rr) - loss_orders
        
        result_info = f"total orders: {len(orders)}, overbought: {overbought}, oversold: {oversold} ,rr: {rr}\nema: {ema_len}, rsi: {rsi_len}, atr: {atr_len}, gain: {gain}\n"
        return {'win_rate': win_rate, 'info': result_info, 'total_order': len(orders), 'rr': rr, 'gain': gain, 'unknow': unknow_result}
    except:
#         print("No orders to action")
        return {'win_rate': 0, 'info': 'No orders to action'}
   

In [6]:
orders = backtest_rsi_o2(df, ema_len=200, rsi_len=14, atr_len=14, overbought=70, oversold=30, rr=1.5)
# print(orders['win_rate'], '%,', orders['info'])
# 69.7 %, total orders: 165, overbought: 69, oversold: 31 ,rr: 1.5
# ema: 200, rsi: 14, atr: 14
print(f"{orders['win_rate']} %, {orders['info']}")

59.26 %, total orders: 81, overbought: 70, oversold: 30 ,rr: 1.5
ema: 200, rsi: 14, atr: 14, gain: 39.0



In [7]:
overB = 95
overS = 5
ema_list = [200] # list(range(100,210, 10))
rsi_list = list(range(10, 20+1))
atr_list = list(range(10, 20+1))
params_list = []

for ema in ema_list:
    for rsi in rsi_list:
        for atr in atr_list:
            for i in range(40):
                params_list.append([ema, rsi, atr, overB-i, overS+i, 1.5])
# params_list[5:10]
len(params_list)

4840

In [8]:
good_params = {'win': 0, 'info': ''}
start = 4000
end = 4840
win_start = 0

loop_num = start
for param in params_list[start:end]:
    result = backtest_rsi_o2(df, ema_len=param[0], rsi_len=param[1], atr_len=param[2], overbought=param[3], oversold=param[4], rr=param[5])
    if result['win_rate'] > win_start and result['total_order'] > 30:
        win_start = result['win_rate']
        good_params['win'] = result['win_rate']
        good_params['info'] = result['info']
    loop_num = loop_num + 1
    print(f"run: {loop_num} to {end}", end = "\r")
    
print(f"\nwin rate: {good_params['win']} %\n{good_params['info']}")

run: 4840 to 4840
win rate: 61.96 %
total orders: 92, overbought: 66, oversold: 34 ,rr: 1.5
ema: 200, rsi: 20, atr: 13, gain: 50.5



In [12]:
# EURUSD 1 pip = 0.0001
# GBPUSD 1 pip = 0.0001
# USDJPY 1 pip = 0.01
# AUDUSD 1 pip = 0.0001
# 7.59 * 0.0001 == 0.000759
0.000759 / 0.01

0.0759

run: 1 to 1000
* win rate: 65.08 %
* total orders: 63, overbought: 74, oversold: 26 ,rr: 1.5
* ema: 200, rsi: 12, atr: 10, gain: 39.5

run: 1001 to 2000
* win rate: 65.62 %
* total orders: 64, overbought: 74, oversold: 26 ,rr: 1.5
* ema: 200, rsi: 12, atr: 15, gain: 38.5

run: 2001 to 3000
* win rate: 63.24 %
* total orders: 68, overbought: 71, oversold: 29 ,rr: 1.5
* ema: 200, rsi: 15, atr: 15, gain: 39.5

run: 3001 to 4000
* win rate: 61.95 %
* total orders: 113, overbought: 65, oversold: 35 ,rr: 1.5
* ema: 200, rsi: 17, atr: 11, gain: 62.0

run: 4801 to 4840
* win rate: 61.96 %
* total orders: 92, overbought: 66, oversold: 34 ,rr: 1.5
* ema: 200, rsi: 20, atr: 13, gain: 50.5

In [20]:
win_rate = 75.61
total_orders = 41
rr = 1.5
win_orders = int((win_rate * total_orders) / 100 )
loss_orders = total_orders - win_orders
gain = (win_orders * rr) - loss_orders
gain

36.5

#### The best params for EURUSD 15m

* win rate: 77.27 %
* total orders: 66, overbought: 76, oversold: 24 ,rr: 1.5
* ema: 200, rsi: 10, atr: 15
* gain 59

#### The best params for GBPUSD 15m

* win rate: 75.61 %
* total orders: 41, overbought: 72, oversold: 28 ,rr: 1.5
* ema: 200, rsi: 17, atr: 10
* OR
* total orders: 41, overbought: 70, oversold: 30 ,rr: 1.5
* ema: 200, rsi: 20, atr: 10
* gain: 36.5

#### The best params for USDJPY 15m

* win rate: 71.43 %
* total orders: 63, overbought: 73, oversold: 27 ,rr: 1.5
* ema: 200, rsi: 12, atr: 17
* gain: 49.5

#### The best params for AUDUSD 15m

* win rate: 61.95 %
* total orders: 113, overbought: 65, oversold: 35 ,rr: 1.5
* ema: 200, rsi: 17, atr: 11
* gain: 62.0

### backtest

In [ ]:
'''
=== Todo ===
[/] import forex data from yohoo finance GBPUSD, EURUSD, USDJPY TF: 5min, 15min, 30min, 1hr, 4hr.
[/] backtest RSI_O2 strategy with forex.
[/] get data from binance BTCUSDT, ETHUSDT, BNBUSDT, ADAUSDT, SHIBUSDT, GALAUSDT, UNIUSDT, CAKEUSDT, SOLUSDT, LUNAUSDT, DOTUSDT
   TF: 15min, 30min, 1hr, 4hr, 1day
[] create EMA_3_lines strategy.
[] backtest EMA_3_lines strategy with forex.
[] create bot pull data from yahoo finance because they can give us last 2 months.
[] Get Free trial 30 days of Tradingview.
'''